## 2.1 针对任意多的分隔符拆分字符串

### 问题

我们需要将字符串拆分为不同的字段，但是分隔符（以及分隔符之间的空格）在整个字符串中并不一致

### 解决方案

字符串对象的 split() 方法只能处理非常简单的情况，而且不支持多个分隔符，对分隔符周围存在的空格也无能为力

所以应该使用 re.split() 方法

In [4]:
import re

line = 'asdf fjdk; afed, fjek,asdf,    foo'
re.split(r'[;,\s]\s*', line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

### 讨论

如果用到了捕获组，那么匹配的文本也会包含在最终结果中

In [2]:
fields = re.split(r'(;|,|\s)\s*', line)
fields

['asdf', ' ', 'fjdk', ';', 'afed', ',', 'fjek', ',', 'asdf', ',', 'foo']

In [3]:
values = fields[::2]
delimiters = fields[1::2] + ['']
print(values)
print(delimiters)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']
[' ', ';', ',', ',', ',', '']


In [4]:
# Reform the line using the same delimiters
''.join(v + d for v, d in zip(values, delimiters))

'asdf fjdk;afed,fjek,asdf,foo'

如果不想在结果中看到分隔字符，但仍然想用括号来对正则表达式模式进行分组，请确保用的是非捕获组

以 (?:...) 的形式指定，详情看 [正则表达式学习参考](https://blog.csdn.net/lxcnn/article/details/4268033)

In [5]:
re.split(r'(?:,|;|\s)\s*', line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

## 2.2 在字符串的开头或结尾处做文本匹配

### 问题

我们需要在字符串的开头或结尾处按照指定的文本模式做检查，例如检查文件的拓展名，URL协议类型等

### 解决方案

str.startswith() 和 str.endswith()

In [6]:
import os
filenames = os.listdir(r'C:\Users\Ph\Desktop\log')
filenames

['Chrome快捷键.jpg',
 'Data Analysis.txt',
 'Git.txt',
 'keyboard-VSCode-windows.pdf',
 'linux.txt',
 'linux常用命令.pdf',
 'Python.txt',
 'sublime text.txt',
 'vi-vim-cheat-sheet-sch.gif',
 'web.txt',
 'windows.txt',
 '手机号解绑.txt']

In [7]:
[name for name in filenames if name.endswith(('.pdf', '.gif'))]

['keyboard-VSCode-windows.pdf', 'linux常用命令.pdf', 'vi-vim-cheat-sheet-sch.gif']

In [9]:
any(name.endswith('.py') for name in filenames)

False

In [10]:
any(name.endswith('.jpg') for name in filenames)

True

In [14]:
choices = ['http:', 'ftp:']
url = 'http://www.python.org'
url.startswith(tuple(choices))  # 需要先用 tuple() 转换成元组

True

## 2.3 利用 Shell 通配符做字符串匹配

### 问题

当工作在 UNIX Shell 下时，我们想使用常见的通配符模式来对文本做匹配

### 解决方案

fnmatch 模块提供了两个函数 —— fnmatch() 和 fnmatchcase() —— 可用来执行这样的匹配

In [10]:
from fnmatch import fnmatch, fnmatchcase
print(fnmatch('foo.txt', '*.txt'))
print(fnmatch('foo.txt', '?oo.txt'))
print(fnmatch('Dat45.csv', 'Dat[0-9]*'))
names = ['Dat1.csv', 'Dat2.csv', 'config.ini', 'foo.py']
[name for name in names if fnmatch(name, 'Dat*.csv')]

True
True
True


['Dat1.csv', 'Dat2.csv']

In [12]:
print(fnmatch('foo.txt', '*.TXT'))
print(fnmatchcase('foo.txt', '*.TXT'))

True
False


In [14]:
addresses = [
    '5412 N CLARK ST',
    '1060 W ADDISON ST',
    '1039 W GRANVILLE AVE',
    '2122 N CLARK ST',
    '4802 N BROADWAY',
]
print([addr for addr in addresses if fnmatchcase(addr, '* ST')])
print([addr for addr in addresses if fnmatchcase(addr, '10[0-9][0-9] W *')])

['5412 N CLARK ST', '1060 W ADDISON ST', '2122 N CLARK ST']
['1060 W ADDISON ST', '1039 W GRANVILLE AVE']


### 讨论

如果只是试着在处理数据时提供一种简单的机制以允许使用通配符，那么通常这都是个合理的解决方案

如果实际上是想编写匹配文件名的代码，那应该使用 glob 模块来完成

## 2.4 文本模式的匹配和查找

### 问题

我们想要按照特定的文本模式进行匹配或查找

### 解决方案

如果想要匹配的只是简单的文字，那么通常只需要用基本的字符串方法

如 str.find()、str.endswith()、str.startswith()

In [15]:
text = 'yeah, but no, but yeah, but no, but yeah'

# Exact match
print(text == 'yeah')

# Match at start or end
print(text.startswith('yeah'))
print(text.endswith('no'))

# Search for the location of the first occurrence
print(text.find('no'))

False
True
False
10


In [16]:
import re

text1 = '11/27/2012'
text2 = 'Nov 27, 2012'

# Simple matching: \d+ means match one or more digits
if re.match(r'\d+/\d+/\d+', text1):
    print('yes')
else:
    print('no')

if re.match(r'\d+/\d+/\d+', text2):
    print('yes')
else:
    print('no')

yes
no


In [17]:
datepat = re.compile(r'\d+/\d+/\d+')
if datepat.match(text1):
    print('yes')
else:
    print('no')

if datepat.match(text2):
    print('yes')
else:
    print('no')

yes
no


match() 方法总是尝试在字符串的开头找到匹配项。如果想针对整个文本搜索出所有的匹配项，那么就应该使用findall()方法

In [18]:
text = 'Today is 3/20/2019. PyCon Starts 3/13/2013'
datepat.findall(text)

['3/20/2019', '3/13/2013']

引入捕获组

In [22]:
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')
m = datepat.match(text1)
m

<re.Match object; span=(0, 10), match='11/27/2012'>

In [25]:
print(m.group(0))
print(m.group(1))
print(m.group(2))
print(m.group(3))
print(m.groups())

11/27/2012
11
27
2012
('11', '27', '2012')


In [26]:
datepat.findall(text)

[('3', '20', '2019'), ('3', '13', '2013')]

In [28]:
for month, day, year in datepat.findall(text):
    print(f'{year}-{month}-{day}')

2019-3-20
2013-3-13


## 2.5 查找和替换文本

### 问题

我们想对字符串中的文本做查找和替换

### 解决方案

对于简单的文本模式，使用 str.replace() 即可
针对更为复杂的模式，可以使用 re 模块中的 sub() 函数/方法

In [1]:
text = 'yeah, but no, but yeah, but no, but yeah'
text.replace('yeah', 'yep')

'yep, but no, but yep, but no, but yep'

In [3]:
import re

text = 'Today is 3/24/2019. PyCon starts 3/13/2013.'
re.sub(r'(\d+)/(\d+)/(\d+)', r'\3-\1-\2', text)

'Today is 2019-3-24. PyCon starts 2013-3-13.'

如果打算用相同的模式执行重复替换，可以考虑先将模式编译以获得更好的性能

In [4]:
import re
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')
datepat.sub(r'\3-\1-\2', text)

'Today is 2019-3-24. PyCon starts 2013-3-13.'

对于更复杂的情况，可以指定一个替换回调函数

In [5]:
from calendar import month_abbr


def change_date(m):
    mon_name = month_abbr[int(m.group(1))]
    return '{} {} {}'.format(m.group(2), mon_name, m.group(3))


datepat.sub(change_date, text)

'Today is 24 Mar 2019. PyCon starts 13 Mar 2013.'

替换回调函数的输入参数是一个匹配对象，由 match() 或 find() 返回。用 .group() 方法来提取匹配中特定的部分。这个函数应该返回替换后的文本

除了得到替换后的文本外，如果还想知道一共完成了多少次替换，可以使用 re.subn()

In [6]:
newtext, n = datepat.subn(r'\3-\1-\2', text)
print(newtext)
print(n)

Today is 2019-3-24. PyCon starts 2013-3-13.
2


## 2.6 以不区分大小写的方式对文本做查找和替换

### 问题

我们需要以不区分大小写的方式在文本中进行查找，可能还需要做替换

### 解决方案

使用 re 模块并且对各种操作都加上 re.IGNORECASE 标记

In [7]:
text = 'UPPER PYTHON, lower python, Mixed Python'
re.findall('python', text, flags=re.IGNORECASE)

['PYTHON', 'python', 'Python']

In [8]:
re.sub('python', 'snake', text, flags=re.IGNORECASE)

'UPPER snake, lower snake, Mixed snake'

上面这个例子揭示出了一种局限，待替换的文本与匹配的文本大小并不吻合

如果想要修正这个问题，需要用到一个支撑函数 ( support function )

In [9]:
def matchcase(word):
    def replace(m):
        text = m.group()
        if text.isupper():
            return word.upper()
        elif text.islower():
            return word.lower()
        elif text[0].isupper():
            return word.capitalize()
        else:
            return word

    return replace


re.sub('python', matchcase('snake'), text, flags=re.IGNORECASE)

'UPPER SNAKE, lower snake, Mixed Snake'

## 2.7 定义实现最短匹配的正则表达式

### 问题

我们正在尝试正则表达式对文本模式做匹配，但识别出来的是最长的可能匹配。相反，我们想将其修改为找出最短的可能匹配

### 解决方案

在模式中的 * 操作符后加上 ? 修饰符以进行非贪婪模式匹配

详情可参考专业正则教程：[正则基础](https://blog.csdn.net/lxcnn/article/category/538256)

In [16]:
str_pat = re.compile(r'\"(.*)\"')
text1 = 'Computer says "no."'
str_pat.findall(text1)

['no.']

In [17]:
text2 = 'Computer says "no." Phone says "yes."'
str_pat.findall(text2)

['no." Phone says "yes.']

In [18]:
str_patt = re.compile(r'\"(.*?)\"')
str_patt.findall(text2)

['no.', 'yes.']

## 2.8 编写多行模式的正则表达式

### 问题

我们打算用正则表达式对一段文本块做匹配，但是希望在进行匹配时能够跨越多行

### 解决方案

我们希望使用句点 (.) 来匹配任意字符，但是句点并不能匹配换行符。要解决这个问题，可以添加对换行符的支持

In [24]:
comment = re.compile(r'/\*(.*?)\*/')
text1 = '/* this is a comment */'
text2 = '''/* this is a 
              multiline comment */
        '''
print(comment.findall(text1))
print(comment.findall(text2))

[' this is a comment ']
[]


In [25]:
comment = re.compile(r'/\*((?:.|\n)*?)\*/')
comment.findall(text2)

[' this is a \n              multiline comment ']

这个模式中，(?:.|\n) 指定了一个非捕获组（即，这个组只做匹配但不捕获结果，也不会分配组号）

其实，在匹配时加 re.S 参数也可多行匹配：

In [35]:
print(re.findall('/\*(.*?)\*/', text2, re.S))
comment = re.compile(r'/\*(.*?)\*/', re.S)
print(comment.findall(text2))

[' this is a \n              multiline comment ']
[' this is a \n              multiline comment ']


### 讨论

re.compile() 函数可以接受一个有用的标记 —— re.DOTALL。这使得正则表达式中的句点(.)可以匹配所有的字符，也包括换行符

In [36]:
comment = re.compile(r'/\*(.*?)\*/', re.DOTALL)
comment.findall(text2)

[' this is a \n              multiline comment ']

## 2.9 将 Unicode 文本统一表示为规范形式

### 问题

我们正在同 Unicode 字符串打交道，但需要确保所有的字符串都拥有相同的底层表示

### 解决方案

使用 unicodedata 模块下的 normalize() 函数

In [37]:
s1 = 'Spicy Jalape\u00f1o'
s2 = 'Spicy Jalapen\u0303o'
print(s1)
print(s2)
print(s1 == s2)
print(len(s1))
print(len(s2))

Spicy Jalapeño
Spicy Jalapeño
False
14
15


In [40]:
from unicodedata import normalize
t1 = normalize('NFC', s1)
t2 = normalize('NFC', s2)
print(t1 == t2)
print(ascii(t1))

True
'Spicy Jalape\xf1o'


In [41]:
t3 = normalize('NFD', s1)
t4 = normalize('NFD', s2)
print(t3 == t4)
print(ascii(t3))

True
'Spicy Jalapen\u0303o'


normalize() 的第一个参数制定了字符串应该如何完成规范表示。NFC表示字符应该是全组成的。NFD表示应该使用组合字符，每个字符应该是能完全分解开的。

Python 还支持 NFKC 和 NFKD 的规范表示形式，它们为处理特定类型的字符增加了额外的兼容功能。

In [44]:
from unicodedata import combining
t1 = normalize('NFD', s1)
''.join(c for c in t1 if not combining(c))

'Spicy Jalapeno'

### 讨论

这个例子展示了 unicodedata 模块的另一个重要功能——用来检测字符是否属于某个字符类别。使用 combining() 函数可对字符做检查，判断它是否为一个组合型字符。这个模块中还有一些函数可用来查找字符类别、检测数字字符等

**参考信息**：

[http://www.unicode.org/faq/normalization.html](http://www.unicode.org/faq/normalization.html)

[https://nedbatchelder.com/text/unipain.html](https://nedbatchelder.com/text/unipain.html)

## 2.10 用正则表达式处理 Unicode 字符

### 问题

我们正在用正则表达式处理文本，但是需要考虑处理 Unicode 字符

### 解决方案

默认情况下 re 模块已经对某些 Unicode 字符类型有了基本的认识

但是，要注意一些特殊情况。例如，当不区分大小写的匹配和大写转换匹配联合起来时，考虑会出现什么行为：

In [45]:
pat = re.compile('stra\u00dfe', re.IGNORECASE)
s = 'straße'
pat.match(s)

<re.Match object; span=(0, 6), match='straße'>

In [46]:
pat.match(s.upper())

In [47]:
s.upper()

'STRASSE'

### 讨论

把 Unicode 和正则表达式混在一起使用绝对是个能让人头痛欲裂的办法。如果真的要这么做，应该考虑安装第三方的正则表达式库（[https://pypi.org/project/regex/](https://pypi.org/project/regex/)）， 这些第三方库针对 Unicode 大写转换提供了完整的支持，还包含其他各种有趣的特性，包括近似匹配

## 2.11 从字符串中去掉不需要的字符

### 问题

我们想在字符串的开始、结尾或中间去掉不需要的字符，比如说空格符

### 解决方案

strip() 方法可用来从字符串的开始和结尾处去掉字符。lstrip() 和 rstrip() 可分别从左或从右侧开始执行去除字符的操作。默认情况下这些方法去除的是空格符，但也可以指定其他的字符

In [56]:
s = '   hello world \n'
print('strip: ', s.strip())
print('lstrip: ', s.lstrip())
print('rstrip: ', s.rstrip())

strip:  hello world
lstrip:  hello world 

rstrip:     hello world


In [57]:
t = '-----hello====='
print(t.lstrip('-'))
print(t.strip('-='))

hello=====
hello


### 讨论

去除字符的操作并不会对位于字符串中间的任何文本起作用。

如果要对里面的空格执行某些操作，应该使用其他技巧，比如 replace() 方法或正则表达式替换

In [59]:
s = '  hello     world    \n'
s = s.strip()
s

'hello     world'

In [61]:
import re
print(s.replace(' ', ''))
print(re.sub('\s+', ' ', s))

helloworld
hello world


In [66]:
with open(r'C:\Users\Ph\Desktop\log\手机号解绑.txt') as f:
    lines = (line.strip() for line in f)
    for line in lines:
        print(line)

App store / Apple ID
PayPal
google
instagram
微信
支付宝
qq
知乎
百度/百度网盘
淘宝
京东
苏宁
微博
领英
钉钉
腾讯云
阿里云
dnspod.cn
知识星球
百度网盘
百度糯米
银行卡
饿了么
美团外卖
滴滴出行
飞猪
虎扑
摩拜单车
西电教务处
bilibili      # 很重要
简书
36氪
网易云音乐
各种邮箱(网易、腾讯、谷歌)
12306火车票订购官网
万方账号 --> 绑在微信上
交管12123




## 2.12 文本过滤和清理

### 问题

某些无聊的脚本小子在 Web 页面表单中填入了 'p̃ỹt̃h̃õñ' 这样的文本，我们想以某种方式将其清理掉

### 解决方案

文本过滤和清理所涵盖的范围非常广泛，设计文本解析和数据处理方面的问题。在非常简单的层次上，我们可能会用基本的字符串函数 (例如 str.upper() 和 str.lower() ) 将文本转换为标准形式。简单的替换操作可通过 str.replace() 或 re.sub() 来完成，它们把重点放在移除或修改特定的字符序列上。也可以利用 unicodedata.normalize() 来规范化文本，如 2.9 节所示。

然而我们可能想更进一步，比方说想清除整个范围内的字符，或者去除音符标志。要完成这些任务，可以使用常被忽视的 str.translate() 方法

In [68]:
s = 'pyth̃on\fis\tawesome\r\n'
s

'pyth̃on\x0cis\tawesome\r\n'

第一步清理空格。要做到这步，先建立一个小型的转换表，然后使用 translate() 方法

In [74]:
remap = {
    ord('\t'): ' ',
    ord('\f'): ' ',
    ord('\r'): None
}
a = s.translate(remap)
a

'pyth̃on is awesome\n'

可以利用这种重新映射的思想进一步构建出更加庞大的转换表。例如，我们把所有的 Unicode 组和字符都去掉：

In [75]:
import unicodedata
import sys
cmb_chrs = dict.fromkeys(c for c in range(sys.maxunicode) if unicodedata.combining(chr(c)))
b = unicodedata.normalize('NFD', a)
b

'pyth̃on is awesome\n'

In [100]:
b.translate(cmb_chrs)

'python is awesome\n'

在这里例子中，我们使用 dict.fromkeys() 方法构建了一个将每个 Unicode 组合字符都映射为 None 的字典

原始输入会通过 unicodedata.normalize() 方法转换为分离形式，然后再通过 translate() 方法删除所有的重音符号

下面看另一个例子。这里有一张转换表将所有的 Unicode 十进制数字字符映射为它们对应的 ASCII 版本

In [101]:
digitmap = { c: ord('0') + unicodedata.digit(chr(c)) for c in range(sys.maxunicode) if unicodedata.category(chr(c)) == 'Nd'}
len(digitmap)

610

In [102]:
x = '\u0661\u0662\u0663'
x.translate(digitmap)

'123'

另一种用来清理文本的技术涉及 I/O 解码和编码函数。大致思路是首先对文本做初步的清理，然后通过结合 encode() 和 decode() 操作来修改或清理文本

下面的例子先对原始文本做分解操作。后续的 ASCII 解码/编码只是简单地一次性丢弃所有不需要的字符。只有当最终目标是 ASCII 形式的文本时才有用

In [104]:
a

'pyth̃on is awesome\n'

In [106]:
b = unicodedata.normalize('NFD', a)
b.encode('ascii', 'ignore').decode('ascii')

'python is awesome\n'

### 讨论

文本过滤和清理的一个主要问题就是运行时的性能。一般来说操作越简单，运行得就越快。对于简单的替换操作，用 str.replace() 通常是最快的方式——即使必须多次调用它也是如此。比方说如果要清理掉空格符：

In [119]:
def clean_spaces(s):
    s = s.replace('\r', '').replace('\t', '').replace('\f', '')
    return s
s = ' py\tt\fh\ron   '
print(s)
print(clean_spaces(s))

on   
 python   


如果试着调用它，就会发现这比使用 translate() 或者正则表达式的方法要快得多。

另一方面，如果需要任何高级的操作，比如字符到字符的重映射或删除，那么 translate() 方法还是非常快的。

## 2.13 对齐文本字符串

### 问题

我们需要以某种对齐方式将文本做格式化处理

### 解决方案

对于基本的字符串对齐要求，可以使用字符串的 ljust()、rjust() 和 center() 方法

In [120]:
text = 'Hello World'
text.ljust(20)

'Hello World         '

In [122]:
text.rjust(20)

'         Hello World'

In [123]:
text.center(20)

'    Hello World     '

In [124]:
text.ljust(20, '=')

'Hello World========='

In [125]:
text.rjust(20, '*')

'*********Hello World'

In [127]:
text.center(20, '_')

'____Hello World_____'

format()函数也可以用来轻松完成对齐的任务。需要做的就是合理利用'<'、'>'，或'^'字符以及一个期望的宽度值

In [128]:
format(text, '>20')

'         Hello World'

In [129]:
format(text, '<20')

'Hello World         '

In [130]:
format(text, '^20')

'    Hello World     '

In [136]:
format(text, '=>20')

'=========Hello World'

In [132]:
format(text, '*<20')

'Hello World*********'

In [133]:
format(text, '_^20')

'____Hello World_____'

In [141]:
'{:>10} {:>10}'.format('Hello', 'World')

'     Hello      World'

format() 的好处之一是它并不是特定于字符串的。它能作用于任何值，这使得它更加通用

In [142]:
x = 1.2345
format(x, '>10')

'    1.2345'

In [143]:
format(x, '^10.2f')

'   1.23   '

### 讨论

在比较老的代码中，通常会发现 % 操作符用来格式化文本

In [145]:
'%-20s' % text

'Hello World         '

In [146]:
'%20s' % text

'         Hello World'

但是在新的代码中，我们应该会更钟情于使用format()函数或方法。format()比 % 操作符提供的功能要强大多了。此外，format()可作用于任何类型的对象，比字符串的 ljust()、rjust() 以及 center() 方法要更加通用。

format()函数的Python在线手册：[https://docs.python.org/3/library/string.html#formatspec](https://docs.python.org/3/library/string.html#formatspec)

## 2.14 字符串拼接及合并

### 问题

我们想将许多小字符串合并成一个大的字符串

### 解决方案

如果想要合并的字符串在一个序列或可迭代对象中，那么将它们合并起来的最快方法就是使用join()方法

In [147]:
pars = ['Is', 'Chicago', 'Not', 'Chicago?']
print(' '.join(pars))
print(','.join(pars))
print(''.join(pars))

Is Chicago Not Chicago?
Is,Chicago,Not,Chicago?
IsChicagoNotChicago?


如果只是想连接一些字符串，一般使用 + 操作符就足够完成任务了

In [148]:
a = 'Is Chicago'
b = 'Not Chicago?'
a + ' ' + b

'Is Chicago Not Chicago?'

如果打算在源代码中将字符串字面值合并在一起，可以简单地将它们排列在一起，中间不加 + 操作符

In [150]:
a = 'Hello' 'World'
a

'HelloWorld'

### 讨论

使用 + 操作符做大量的字符串拼接是非常低效的，最好先收集所有要连接的部分，最后再一次通过 join() 将它们连接起来

一个相关的技巧（很漂亮的技巧）是利用生成器表达式

In [151]:
data = ['ACME', 50, 91.1]
','.join(str(d) for d in data)

'ACME,50,91.1'

In [159]:
a = 'Is Amoy'
b = 'Not Amoy?'
c = 'Must Amoy'
print(a + ' : ' + b + ' : ' + c)  # Ugly
print(' : '.join([a, b, c]))      # Still Ugly
print(a,b,c, sep=' : ')           # Better

Is Amoy : Not Amoy? : Must Amoy
Is Amoy : Not Amoy? : Must Amoy
Is Amoy : Not Amoy? : Must Amoy


如果我们编写的代码要从许多短字符串中构建输出，则应该考虑编写生成器函数

In [163]:
def sample():
    s = 'Is Amoy Not Amoy?'.split()
    for i in s:
        yield i
text = ' '.join(sample())
print(text)

Is Amoy Not Amoy?


可以将这些片段重定向到 I/O：

又或者我们能以混合的方式将 I/O 操作智能化地结合在一起：

## 2.15 给字符串中的变量名做插值处理

### 问题

我们想创建一个字符串，其中嵌入的变量名称会以变量的字符串值形式替换掉

### 解决方案

Python 并不直接支持在字符串中对变量做简单的值替换。但是，这个功能可以通过字符串的 format() 方法近似模拟出来

In [167]:
s = '{name} has {n} messages.'
s.format(name='Amoy', n=37)

'Amoy has 37 messages.'

如果要被替换的值确实能在变量中找到，则可以将 format_map() 和 vars() 联合起来使用

In [168]:
name = 'Amoy'
n = 37
s.format_map(vars())

'Amoy has 37 messages.'

Python3.6 版本开始，有了一个更优雅的解决方案： f-string

并且 f-string 也可以自定义格式：对齐、宽度、符号、补零、精度、进制等  
参考博客：[https://blog.csdn.net/sunxb10/article/details/81036693](https://blog.csdn.net/sunxb10/article/details/81036693)

In [176]:
f'{name} has {n} messages.'

'Amoy has 37 messages.'

In [177]:
F'{name} has {n} messages.'

'Amoy has 37 messages.'

In [174]:
f'{7 * 3 + 4}'

'25'

In [175]:
f'{name.lower()} has {n} messages.'

'amoy has 37 messages.'

### 三种变量名插值处理的效率测试

In [179]:
timeit '%s has %s messages.' %(name, n)

408 ns ± 1.88 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [180]:
timeit '{name} has {n} message.'.format(name=name, n=n)

777 ns ± 2.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [181]:
timeit '{} has {} message'.format(name, n)

486 ns ± 1.34 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [182]:
timeit f'{name} has {n} message.'

256 ns ± 0.879 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


可见 f-string 的语法是**效率最高**的

## 2.16 以固定的列数重新格式化文本

### 问题

我们有一些很长的字符串，想将它们重新格式化，使得它们能按照用户指定的列数来显示

### 解决方案

可以用 textwrap 模块来重新格式化文本的输出

In [190]:
import textwrap
s = "Plotly's Python graphing library makes interactive, \
     publication-quality graphs online. \
     Tutorials and tips about fundamental features of Plotly's python API."
print(textwrap.fill(s, 105), '\n')
print(textwrap.fill(s, 48), '\n')
print(textwrap.fill(s, 48, initial_indent='    '), '\n')
print(textwrap.fill(s, 48, subsequent_indent='    '))

Plotly's Python graphing library makes interactive,      publication-quality graphs online.
Tutorials and tips about fundamental features of Plotly's python API. 

Plotly's Python graphing library makes
interactive,      publication-quality graphs
online.      Tutorials and tips about
fundamental features of Plotly's python API. 

    Plotly's Python graphing library makes
interactive,      publication-quality graphs
online.      Tutorials and tips about
fundamental features of Plotly's python API. 

Plotly's Python graphing library makes
    interactive,      publication-quality graphs
    online.      Tutorials and tips about
    fundamental features of Plotly's python API.


### 讨论

textwrap 模块能够以简单直接的方式对文本格式做整理使其适合打印——尤其是当希望输出结果能很好地显示在终端上时。关于终端的尺寸大小，可以通过 os.get_terminal_size() 来获取

In [189]:
import os
os.get_terminal_size().columns

105

## 2.17 在文本中处理 HTML 和 XML 实体

### 问题

我们想将 &entity 或 &#code 这样的 HTML 或 XML 实体替换为它们相对应的文本。或者，我们需要生成文本，但是要对特定的字符（比如<,>或&）做转义处理

### 解决方案

如果要生成文本，使用 html.escape() 函数来完成替换 <or\> 这样的特殊字符相对来说是比较容易的

In [191]:
s = 'Elements are written as "<tag>text</tag>".'
import html
print(s)

Elements are written as "<tag>text</tag>".


In [192]:
print(html.escape(s))

Elements are written as &quot;&lt;tag&gt;text&lt;/tag&gt;&quot;.


In [193]:
# Disable escaping of quotes
print(html.escape(s, quote=False))

Elements are written as "&lt;tag&gt;text&lt;/tag&gt;".


生成 ASCII 文本：

In [195]:
s = 'Spicy Jalapeño'
s.encode('ascii', errors='xmlcharrefreplace')

b'Spicy Jalapen&#771;o'

如果由于某种原因在得到的文本中带有一些实体，而我们想手工将它们替换掉，通常可以利用各种 HTML 或 XML 解析器自带的功能函数和方法来完成

In [199]:
import html
s = 'Spicy &quot;Jalape&#241;o&quot.'
html.unescape(s)

'Spicy "Jalapeño".'

In [200]:
from xml.sax.saxutils import unescape
t = 'The prompt is &gt;&gt;&gt;'
unescape(t)

'The prompt is >>>'

### 讨论

在生成 HTML 或 XML 文档时，适当地对特殊字符做转义处理常常是个容易被忽视的细节。尤其是当自己用 print() 或其他一些基本的字符串格式化函数来产生这类输出时更是如此。简单的解决方案是使用像 html.escape() 这样的工具函数

如果需要反过来处理文本（即，将 HTML 或 XML 实体转换成对应的字符），有许多像 xml.sax.saxutils.unescape() | html.unescape() 这样的工具函数能帮上忙。

如果是处理 HTML 或 XML，像 html.parser | xml.etree.ElementTree | lxml.etree.HTML 这样的解析模块应该已经解决了有关替换文本中实体的细节问题

## 2.18 文本分词

### 问题

我们有一个字符串，想从左到右将它解析为标记流（ stream of tokens ）

### 解决方案

利用模式对象的 scanner() 方法来完成分词操作。该方法会创建一个扫描对象，在给定的文本中重复调用 match() ，一次匹配一个模式

In [3]:
import re
from collections import namedtuple
NAME = r'(?P<NAME>[a-zA-Z_][a-zA-Z_0-9]*)'
NUM = r'(?P<NUM>\d+)'
PLUS = r'(?P<PLUS>\+)'
TIMES = r'(?P<TIMES>\*)'
EQ = r'(?P<EQ>=)'
WS = r'(?P<WS>\s+)'
master_pat = re.compile('|'.join([NAME, NUM, PLUS, TIMES, EQ, WS]))
text = 'foo = 23 + 42 * 10'
Token = namedtuple('Token', ['type', 'value'])

def generate_tokens(pat, text):
    scanner = pat.scanner(text)
    for m in iter(scanner.match, None):
        yield Token(m.lastgroup, m.group())
        
for tok in generate_tokens(master_pat, text):
    print(tok)

Token(type='NAME', value='foo')
Token(type='WS', value=' ')
Token(type='EQ', value='=')
Token(type='WS', value=' ')
Token(type='NUM', value='23')
Token(type='WS', value=' ')
Token(type='PLUS', value='+')
Token(type='WS', value=' ')
Token(type='NUM', value='42')
Token(type='WS', value=' ')
Token(type='TIMES', value='*')
Token(type='WS', value=' ')
Token(type='NUM', value='10')


如果想以某种方式对标记流做过滤处理，可以用生成器表达式。比如过滤掉所有的空格标记：

In [9]:
tokens = (tok for tok in generate_tokens(master_pat, text) if tok.type != 'WS')
for tok in tokens:
    print(tok)

Token(type='NAME', value='foo')
Token(type='EQ', value='=')
Token(type='NUM', value='23')
Token(type='PLUS', value='+')
Token(type='NUM', value='42')
Token(type='TIMES', value='*')
Token(type='NUM', value='10')


### 讨论

对于更加高级的文本解析，第一步往往是分词处理。这里推荐三个中文分词的包：

1. 要“做最好的中文分词组件”的**结巴分词**。[https://github.com/fxsjy/jieba](https://github.com/fxsjy/jieba)
2. 来自清华的**THULAC**。[https://github.com/thunlp/THULAC-Python](https://github.com/thunlp/THULAC-Python)
3. 来自北大的**PKUSeg**。[https://github.com/lancopku/pkuseg-python](https://github.com/lancopku/pkuseg-python)

## 2.19 编写一个简单的递归下降解析器

### 问题

我们需要根据一组语法规则来解析文本，以此执行相应的操作或构建一个抽象语法树来表示输入

### 解决方案

在这个问题中，我们把重点放在根据特定的语法来解析文本上。要做到这些，应该以 BNF 或 EBNF 的形式定义出语法的正是规格

下面是构建一个递归下降的表达式计算器的Demo：

In [29]:
import re
import collections

# Token specification
NUM = r'(?P<NUM>\d+)'
PLUS = r'(?P<PLUS>\+)'
MINUS = r'(?P<MINUS>-)'
TIMES = r'(?P<TIMES>\*)'
DIVIDE = r'(?P<DIVIDE>/)'
LPAREN = r'(?P<LPAREN>\()'
RPAREN = r'(?P<RPAREN>\))'
WS = r'(?P<WS>\s+)'

master_pat = re.compile('|'.join([NUM, PLUS, MINUS, TIMES, DIVIDE, LPAREN, RPAREN, WS]))

# Tokenizer
Token = collections.namedtuple('Token', ['type', 'value'])

def generate_tokens(text):
    scanner = master_pat.scanner(text)
    for m in iter(scanner.match, None):
        tok = Token(m.lastgroup, m.group())
        if tok.type != 'WS':
            yield tok
            
# Parser
class ExpressionEvaluator:
    def parse(self, text):
        self.tokens = generate_tokens(text)
        self.tok = None    # Last symbol consumed
        self.nexttok = None    # Next symbok tokenized
        self._advance()    # Load first lookahead token
        return self.expr()

    def _advance(self):
        'Advence one token ahead'
        self.tok, self.nexttok = self.nexttok, next(self.tokens, None)
    
    def _accept(self, toktype):
        'Test and consume the next token if it matches toktype'
        if self.nexttok and self.nexttok.type == toktype:
            self._advance()
            return True
        else:
            return False
        
    def _expect(self, toktype):
        'Consume next token if it matches toktype or raise SyntaxError'
        if not self._accept(toktype):
            raise SyntaxError('Expected ' + toktype)
            
    # Grammar rules follow
    
    def expr(self):
        "expression ::= term { ('+'|'-') term }*"
        
        exprval = self.term()
        while self._accept('PLUS') or self._accept('MINUS'):
            op = self.tok.type
            right = self.term()
            if op == 'PLUS':
                exprval += right
            elif op == 'MINUS':
                exprval -= right
        return exprval
    
    def term(self):
        "term ::= factor { ('*'|'/') factor }*"
        
        termval = self.factor()
        while self._accept('TIMES') or self._accept('DIVIDE'):
            op = self.tok.type
            right = self.factor()
            if op == 'TIMES':
                termval *= right
            elif op == 'DIVIDE':
                termval /= right
        return termval
    
    def factor(self):
        'factor ::= NUM | ( expr )'
        
        if self._accept('NUM'):
            return int(self.tok.value)
        elif self._accept('LPAREN'):
            exprval = self.expr()
            self._expect('RPAREN')
            return exprval
        else:
            raise SyntaxError('Expected NUMBER or LPAREN')

下面是以交互的方式使用 ExpressionEvaluator 类的示例：

In [30]:
e = ExpressionEvaluator()
e.parse('2')

2

In [10]:
e.parse('2 + 3')

5

In [11]:
e.parse('2 + 3 * 4')

14

In [12]:
e.parse('2 + (3 + 4) * 5')

37

In [33]:
e.parse('2 + 8 / 4')

4.0

In [13]:
e.parse('2 + (3 + * 4)')

SyntaxError: Expected NUMBER or LPAREN (<string>)

### 解析树Demo

如果我们想做的不只是纯粹的计算，那就需要修改 ExpressionEvaluator 类来实现

比如，下面的实现构建了一棵简单的解析树

In [14]:
class ExpressionTreeBuilder(ExpressionEvaluator):
    def expr(self):
        "expression ::= term { ('+'|'-') term }"
        
        exprval = self.term()
        while self._accept('PLUS') or self._accept('MINUS'):
            op = self.tok.type
            right = self.term()
            if op == 'PLUS':
                exprval = ('+', exprval, right)
            elif op == 'MINUS':
                exprval = ('-', exprval, right)
        return exprval
    
    def term(self):
        "term ::= factor { ('*'|'/') factor }"
        
        termval = self.factor()
        while self._accept('TIMES') or self._accept('DIVIDE'):
            op = self.tok.type
            right = self.factor()
            if op == 'TIMES':
                termval = ('*', termval, right)
            elif op == 'DIVIDE':
                termval = ('/', termval, right)
            return termval
        
    def factor(self):
        'factor ::= NUM | ( expr )'
        
        if self._accept('NUM'):
            return int(self.tok.value)
        elif self._accept('LPRREN'):
            exprval = self.expr()
            self._expect('RPAREN')
            return exprval
        else:
            raise SyntaxError('Expected NUMBER or LPAREN')

In [39]:
e = ExpressionTreeBuilder()

# 这个小bug还没找出来
e.parse('5 + 4')

('+', None, None)

In [35]:
e.parse('5 * 4')

('*', 5, 4)

In [36]:
e.parse('6 / 3')

('/', 6, 3)

## 2.20 在字符串上执行文本操作

### 问题

我们想在字节串（ Byte String ）上执行常见的文本操作（例如，拆分、搜索和替换）

### 解决方案

字节串已经支持大多数和文本字符串一样的内建操作

In [10]:
data = b'Hello World'
data[0:5]

b'Hello'

In [11]:
data.startswith(b'Hello')

True

In [12]:
data.split()

[b'Hello', b'World']

In [13]:
data.replace(b'Hello', b'Hello Cruel')

b'Hello Cruel World'

也可以在字节串上执行正则表达式的模式匹配操作

In [15]:
import re
data = b'FOO:BAR,SPAM'
re.split(b'[:,]', data)

[b'FOO', b'BAR', b'SPAM']

### 讨论

就绝大部分情况而言，几乎所有能在文本字符串上执行的操作同样也可以在字节串上进行。但是，还是有几个显著的区别

In [16]:
a = 'Hello World'  # Text string
a[0]

'H'

In [17]:
b = b'Hello World'
b[0]

72

同理，在字节串上是没有普通字符串那样的格式化操作的。  
如果想在字节串上做任何形式的格式化操作，应该使用普通的文本字符串然后再做编码

In [22]:
'{:7s}{:7d}{:7.2f}'.format('ACME', 100, 490.1).encode('ascii')

b'ACME       100 490.10'

最后，由于字节串和 Python 中许多其他部分并不能很好地相容，这样为了保证结果的正确性，我们只能手动去执行各种各样的编码/解码操作。  
坦白地说，如果要同文本打交道，在程序中使用普通的文本字符串就好，不要用字节串。